In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote
from PIL import Image

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash

In [2]:
commons_site = pb.Site("commons", "commons")

In [3]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [15]:
#### User input:
url = 'https://diario.madrid.es/blog/notas-de-prensa/villa-de-vallecas-dispone-ya-de-un-nuevo-rocodromo/'

user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'
headers = {'User-Agent' : user_agent}

#### User input:
categories = ['Climbing walls in Madrid', 
              'Sports venues in Villa de Vallecas district', 
              'Ensanche de Vallecas']

In [16]:
#### User input:
upload_categories = ['Files uploaded by User:Discasto']
categories = categories + upload_categories
categories

['Climbing walls in Madrid',
 'Sports venues in Villa de Vallecas district',
 'Ensanche de Vallecas',
 'Files uploaded by User:Discasto']

In [17]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
title

'Villa de Vallecas dispone ya de un nuevo rocódromo'

In [19]:
date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
#date = ""
year = date.split('-')[0]
month = calendar.month_name[int(date.split('-')[1])]
date

'2018-01-23'

In [20]:
body = soup.find_all("div", class_="post-content")
p_description = body[0].find_all("p")
for p in p_description :
    if len(p.get_text()) > 10:
        description = p.get_text()
        break
#description=u'Una delegación del Ayuntamiento de Madrid asiste a la manifestación de Barcelona en contra de los atentados terroristas, presidida por Manuela Carmena, y con representación de todos los grupos políticos municipales. Esta representación municipal ha estado compuesta por la portavoz del gobierno municipal, Rita Maestre: el delegado de Coordinación Territorial y Cooperación Público-Social y concejal del Grupo Municipal Ahora Madrid, Nacho Murgui; la concejala y delegada del portavoz del Grupo Municipal Partido Popular, Isabel Rosell; el concejal y delegado de la portavoz del Grupo Municipal Socialista, Ignacio de Benito y la portavoz del Grupo Municipal Ciudadanos-Partido de la Ciudadanía, Begoña Villacís.\n"Madrid como ciudad está aquí, con Barcelona, por solidaridad y por la necesidad de expresar la protesta que significan los actos de esta barbarie, para hacer posible que no se vuelvan a repetir", ha dicho la alcaldesa a su llegada a la capital catalana.'
description

'El concejal de los distritos de Puente de Vallecas y Villa de Vallecas, Francisco Pérez Ramos, ha inaugurado esta mañana un rocódromo que se sitúa en la avenida de La Gavia, en el Ensanche de Vallecas.'

In [21]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:${month} ${year} in Madrid]]
${cat_string}"""

vars = {
    "url": url,
    "description": description,
    "year": year,
    "month": month,
    "date": date,
    "title": title,
    "cat_string": '\n'.join(['[[Category:'+i+']]' for i in categories])
}
t = Template(template)
_text = t.render(**vars)
_text

'=={{int:filedesc}}==\n{{Information\n|description={{es|1=El concejal de los distritos de Puente de Vallecas y Villa de Vallecas, Francisco Pérez Ramos, ha inaugurado esta mañana un rocódromo que se sitúa en la avenida de La Gavia, en el Ensanche de Vallecas.}}\n|date=2018-01-23\n|source=[https://diario.madrid.es/blog/notas-de-prensa/villa-de-vallecas-dispone-ya-de-un-nuevo-rocodromo/ Diario de Madrid - Villa de Vallecas dispone ya de un nuevo rocódromo]\n|author=[https://diario.madrid.es/ Diario de Madrid]\n|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]\n|other versions=\n}}\n\n=={{int:license-header}}==\n{{Diario de Madrid}}\n\n[[Category:Images from Ayuntamiento de Madrid (to classify)]]\n[[Category:January 2018 in Madrid]]\n[[Category:Climbing walls in Madrid]]\n[[Category:Sports venues in Villa de Vallecas district]]\n[[Category:Ensanche de Vallecas]]\n[[Category:Files uploaded by User:Discasto]]'

In [25]:
images = [image.a["href"] for image in soup.find_all("div", class_="gallery-icon")]
images

['https://diario.madrid.es/wp-content/uploads/2018/01/Inauguración-Rocódromo-2-562x1000.jpg',
 'https://diario.madrid.es/wp-content/uploads/2018/01/Inauguración-Rocódromo-4-1500x842.jpg',
 'https://diario.madrid.es/wp-content/uploads/2018/01/Inauguración-Rocódromo-1-562x1000.jpg']

In [23]:
#### User input:
excluded = []

used_names = []
counter = 0

for image in images :
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
        
    # First, the image is downloaded and stored
    image_url = quote(image.encode('utf-8'), ':/')
    image_name = '{} {:02d}.jpg'.format(title.replace(':', ' -').replace('  ', ' '), counter+1)
    image_path = os.path.join(images_directory, image_name)
    try: 
        r = requests.get(image_url, headers=headers)
        image = Image.open(BytesIO(r.content))
        image.save(image_path)
    except :
        print ('Failed download. Skipping')
        continue

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)
    counter += 1

Preparing to upload image with name Villa de Vallecas dispone ya de un nuevo rocódromo 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El concejal de los distritos de Puente de Vallecas y Villa de Vallecas, Francisco Pérez Ramos, ha inaugurado esta mañana un rocódromo que se sitúa en la avenida de La Gavia, en el Ensanche de Vallecas.}}
|date=2018-01-23
|source=[https://diario.madrid.es/blog/notas-de-prensa/villa-de-vallecas-dispone-ya-de-un-nuevo-rocodromo/ Diario de Madrid - Villa de Vallecas dispone ya de un nuevo rocódromo]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:January 2018 in Madrid]]
[[Category:Climbing walls in Madrid]]
[[Category:Sports venues in Villa de Vallecas district]]
[[Category:Ensanche de Vallecas]]
[[Category:Files uploaded by User:Discasto]]
Upload

Preparing to upload image with name Villa de Vallecas dispone ya de un nuevo rocódromo 02.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El concejal de los distritos de Puente de Vallecas y Villa de Vallecas, Francisco Pérez Ramos, ha inaugurado esta mañana un rocódromo que se sitúa en la avenida de La Gavia, en el Ensanche de Vallecas.}}
|date=2018-01-23
|source=[https://diario.madrid.es/blog/notas-de-prensa/villa-de-vallecas-dispone-ya-de-un-nuevo-rocodromo/ Diario de Madrid - Villa de Vallecas dispone ya de un nuevo rocódromo]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:January 2018 in Madrid]]
[[Category:Climbing walls in Madrid]]
[[Category:Sports venues in Villa de Vallecas district]]
[[Category:Ensanche de Vallecas]]
[[Category:Files uploaded by User:Discasto]]
Upload

Preparing to upload image with name Villa de Vallecas dispone ya de un nuevo rocódromo 03.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El concejal de los distritos de Puente de Vallecas y Villa de Vallecas, Francisco Pérez Ramos, ha inaugurado esta mañana un rocódromo que se sitúa en la avenida de La Gavia, en el Ensanche de Vallecas.}}
|date=2018-01-23
|source=[https://diario.madrid.es/blog/notas-de-prensa/villa-de-vallecas-dispone-ya-de-un-nuevo-rocodromo/ Diario de Madrid - Villa de Vallecas dispone ya de un nuevo rocódromo]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=[https://diario.madrid.es/contenidos-libres/ License information for all contents in diario.madrid.es]
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:January 2018 in Madrid]]
[[Category:Climbing walls in Madrid]]
[[Category:Sports venues in Villa de Vallecas district]]
[[Category:Ensanche de Vallecas]]
[[Category:Files uploaded by User:Discasto]]
Upload